In [2]:
import sys, os, pathlib

print("Python:", sys.version)
print("CWD:", os.getcwd())

# ใช้ relative path (ไม่มี leading slash) หรือใช้ os.path.join กับ cwd
IDF_PATH = os.path.join(os.getcwd(), "EnergyPlus_BP_Boonchoo", "2ZoneDataCenterHVAC_wEconomizer.idf")
EPW_PATH = os.path.join(os.getcwd(), "EnergyPlus_BP_Boonchoo", "DEU_BW_Stuttgart.AP.107380_TMYx.2009-2023.epw")

OUT_DIR = "eplusout"
pathlib.Path(OUT_DIR).mkdir(exist_ok=True)

# ตรวจสอบว่าไฟล์เจอหรือไม่
print("IDF exists:", os.path.exists(IDF_PATH))
print("EPW exists:", os.path.exists(EPW_PATH))


Python: 3.10.10 (main, Aug  4 2023, 20:19:42) [GCC 9.3.0 20200312 (Cray Inc.)]
CWD: /lustrefs/disk/project/lt200291-ignite/Project_chomwong/project
IDF exists: True
EPW exists: True


In [3]:
from collections import deque
from datetime import datetime
import json, uuid

import sys
sys.path.append("/project/lt200291-ignite/Project_chomwong/energyplus/EnergyPlus-25.1.0-1c11a3d85f-Linux-CentOS7.9.2009-x86_64")  # ตัวอย่าง path, ดูจริงด้วย `module show energyplus`
from pyenergyplus.api import EnergyPlusAPI

from pyenergyplus.api import EnergyPlusAPI

# คอนฟิก co-sim
ZONE_NAME = "West Zone"                # ปรับให้ตรงกับ IDF
COOL_SP_ACTUATOR = ("Schedule:Constant", "Cooling Setpoint", ZONE_NAME)
HEAT_SP_ACTUATOR = ("Schedule:Constant", "Heating Setpoint", ZONE_NAME)

BATCH_SIZE = 1000


In [4]:
from mesa import Model, Agent
from mesa.time import RandomActivation

class Occupant(Agent):
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.comfort = 0.0
    def step(self):
        Tin = self.model.shared_state.get("Tin", 25.0)
        self.comfort = max(0, 1 - abs(Tin - 24)/6)

class BuildingABM(Model):
    def __init__(self, n_agents: int, shared_state: dict):
        super().__init__()
        self.schedule = RandomActivation(self)
        self.shared_state = shared_state
        for i in range(n_agents):
            self.schedule.add(Occupant(i, self))
        self.cool_sp = 24.0
        self.heat_sp = 20.0

    def decide_actions(self):
        comforts = [a.comfort for a in self.schedule.agents]
        avg_c = sum(comforts)/len(comforts) if comforts else 1.0
        Tin = self.shared_state.get("Tin", 25.0)
        if avg_c < 0.8 and Tin > self.cool_sp:
            self.cool_sp -= 0.2
        self.cool_sp = max(22.0, min(26.0, self.cool_sp))
        return {"cool_sp": self.cool_sp, "heat_sp": self.heat_sp}

    def step(self):
        self.schedule.step()
        return self.decide_actions()


In [5]:
class InMemoryCollector:
    def __init__(self, batch_size=500):
        self.run_id = str(uuid.uuid4())
        self.buffer = deque()
        self.batch_size = batch_size
        self.file_index = 0

    def append(self, row: dict):
        self.buffer.append(row)
        if len(self.buffer) >= self.batch_size:
            self.flush()

    def flush(self):
        if not self.buffer:
            return
        try:
            import pandas as pd
            import pyarrow as pa, pyarrow.parquet as pq
            df = pd.DataFrame(list(self.buffer))
            table = pa.Table.from_pandas(df, preserve_index=False)
            pq.write_table(table, f"{OUT_DIR}/run_{self.run_id}_{self.file_index:03d}.parquet")
            self.file_index += 1
            self.buffer.clear()
        except Exception:
            with open(f"{OUT_DIR}/run_{self.run_id}_{self.file_index:03d}.csv", "a", encoding="utf-8") as f:
                while self.buffer:
                    r = self.buffer.popleft()
                    f.write(json.dumps(r, ensure_ascii=False) + "\n")
            self.file_index += 1


In [6]:
class EPHandles:
    def __init__(self):
        self.var = {}
        self.act = {}

def get_var_handle(api, state, cache: EPHandles, var_name, key):
    k = (var_name, key)
    h = cache.var.get(k)
    if h is None:
        h = api.exchange.get_variable_handle(state, var_name, key)
        cache.var[k] = h
    return h

def get_var_value(api, state, cache: EPHandles, var_name, key):
    h = get_var_handle(api, state, cache, var_name, key)
    if h == -1:
        return None
    return api.exchange.get_variable_value(state, h)

def get_act_handle(api, state, cache: EPHandles, act_type, act_name, key):
    k = (act_type, act_name, key)
    h = cache.act.get(k)
    if h is None:
        h = api.exchange.get_actuator_handle(state, act_type, act_name, key)
        cache.act[k] = h
    return h

def set_act_value(api, state, cache: EPHandles, act_type, act_name, key, value):
    h = get_act_handle(api, state, cache, act_type, act_name, key)
    if h not in (None, -1):
        api.exchange.set_actuator_value(state, h, float(value))


In [7]:
class CoSimSession:
    def __init__(self, idf_path, epw_path, out_dir, zone_name, n_agents=25, batch_size=1000):
        self.api = EnergyPlusAPI()
        self.state = self.api.state_manager.new_state()
        self.zone = zone_name
        self.shared_state = {}
        self.abm = BuildingABM(n_agents=n_agents, shared_state=self.shared_state)
        self.collector = InMemoryCollector(batch_size=batch_size)
        self.handles = EPHandles()
        self._cb_registered = False
        self.idf = idf_path
        self.epw = epw_path
        self.out_dir = out_dir

    # ---- callback ----
    def on_begin_zone_timestep(self, state_argument) -> None:
        year = self.api.exchange.year(state_argument)
        doy = self.api.exchange.day_of_year(state_argument)
        hour = self.api.exchange.hour(state_argument)
        minute = self.api.exchange.minutes(state_argument)

        # ดึงค่าจาก EnergyPlus
        Tin  = get_var_value(self.api, state_argument, self.handles, "Zone Air Temperature", self.zone)
        Tout = get_var_value(self.api, state_argument, self.handles, "Site Outdoor Air Drybulb Temperature", "ENVIRONMENT")
        Pel  = get_var_value(self.api, state_argument, self.handles, "Facility Total Electricity Demand Rate", "WHOLE BUILDING")

        # ใส่ default value ถ้า None
        Tin  = Tin  if Tin  is not None else 25.0
        Tout = Tout if Tout is not None else 30.0
        Pel  = Pel  if Pel  is not None else 0.0
        
        # optional debug print
        print("Tin:", Tin, "Tout:", Tout, "Pel:", Pel)

        # update shared state
        self.shared_state.update({"Tin": Tin, "Tout": Tout, "P_el": Pel})

        # เรียก ABM step()
        actions = self.abm.step()

        # ส่งค่าไป EnergyPlus
        set_act_value(self.api, state_argument, self.handles, *COOL_SP_ACTUATOR, value=actions.get("cool_sp", 24.0))
        set_act_value(self.api, state_argument, self.handles, *HEAT_SP_ACTUATOR, value=actions.get("heat_sp", 20.0))

        # เก็บผลลัพธ์
        row = {
            "run_id": self.collector.run_id,
            "year": year, "doy": doy, "hour": hour, "minute": minute,
            "zone": self.zone,
            "Tin": Tin, "Tout": Tout, "P_el": Pel,
            "cool_sp": actions.get("cool_sp", 24.0),
            "heat_sp": actions.get("heat_sp", 20.0),
            "n_agents": len(self.abm.schedule.agents),
            "avg_comfort": sum(a.comfort for a in self.abm.schedule.agents)/max(1, len(self.abm.schedule.agents)),
            "ts_wallclock": datetime.utcnow().isoformat()
        }
        self.collector.append(row)

    def register_callbacks(self):
        if not self._cb_registered:
            self.api.runtime.callback_begin_zone_timestep_after_init_heat_balance(self.state, self.on_begin_zone_timestep)
            self._cb_registered = True

    def run(self):
        self.register_callbacks()
        self.api.runtime.run_energyplus(self.state, [
            "-w", self.epw,
            "-d", self.out_dir,
            self.idf
        ])
        self.collector.flush()


In [8]:
session = CoSimSession(IDF_PATH, EPW_PATH, OUT_DIR, ZONE_NAME, n_agents=25, batch_size=BATCH_SIZE)
session.run()

EnergyPlus Starting
EnergyPlus, Version 25.1.0-1c11a3d85f, YMD=2025.10.12 21:57
Initializing Response Factors
Calculating CTFs for "EXT-WALLS"
Calculating CTFs for "FLOOR"
Calculating CTFs for "PARTITION"
Calculating CTFs for "PLENUM FLOOR"
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Computing Interior Solar Absorption Factors
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Tin: 25.0 Tout: -17.3 Pel: 0.0
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Window Shading
Computing Interior Absorption Factors
Computing Interior Diffuse Solar Absorption Factors
Initializing Solar Heat Gains
Initializing Internal Heat Gains
Initializing Interior Solar Di

EnergyPlus Completed Successfully.


In [65]:
import glob
import pandas as pd
import os

# สร้าง path output
OUT_CSV = os.path.join(OUT_DIR, "outCosimulation.csv")

# ดึงไฟล์ parquet ก่อน
parqs = sorted(glob.glob(f"{OUT_DIR}/run_{session.collector.run_id}_*.parquet"))
if parqs:
    df = pd.concat([pd.read_parquet(p) for p in parqs], ignore_index=True)
else:
    # ถ้าไม่มี parquet ใช้ csv / json log
    logs = sorted(glob.glob(f"{OUT_DIR}/run_{session.collector.run_id}_*.csv"))
    if logs:
        df = pd.concat([pd.read_csv(p) for p in logs], ignore_index=True)
    else:
        # ไม่มีไฟล์เลย → ใช้ data จาก collector โดยตรง
        df = pd.DataFrame(session.collector.data)

# export เป็น CSV ชื่อ outCosimulation.csv
df.to_csv(OUT_CSV, index=False, encoding="utf-8")

print("Exported CSV to:", OUT_CSV)


Exported CSV to: eplusout/outCosimulation.csv
